In [1]:
import pandas as pd

df = pd.read_csv("data.csv")
second_col = df.iloc[:, 1]
distinct_values = pd.unique(second_col.dropna())

print(distinct_values)
print("No of Regions: ", len(distinct_values))

['BBI' 'CCU' 'GAU' 'PAT' 'EUP' 'GGN' 'HHPT' 'JAI' 'NDL' 'SDL' 'WUP' 'BLR'
 'CHE' 'CJB' 'COK' 'HYD' 'VJA' 'AMD' 'IDR' 'MUM' 'NGP' 'PNQ']
No of Regions:  22


/var/folders/lw/mxt4vf712k5_lslhrjcqqj700000gn/T/ipykernel_67244/811402059.py:3: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,

In [ ]:
import pandas as pd

# ==========================
# Step 1: Read CSV (skip empty first row)
# ==========================
raw = pd.read_csv("data.csv", header=None, skiprows=1)

# ==========================
# Step 2: Extract metadata
# ==========================
types = raw.iloc[0, 7:].astype(str).str.strip().tolist()          # type row
branch_codes = raw.iloc[4, 7:].astype(str).str.strip().tolist()   # branch row
modes = raw.iloc[6:, 5].astype(str).str.strip().tolist()          # mode column
des_regions = raw .iloc[6:, 1].astype(str).str.strip().tolist()    # region column

# Numeric data
data_values = raw.iloc[6:, 7:].apply(pd.to_numeric, errors='coerce').fillna(0)

# ==========================
# Step 3: Build long-format DataFrame
# ==========================
rows = []
for i, (region, mode) in enumerate(zip(des_regions, modes)):
    for j, (branch, type_) in enumerate(zip(branch_codes, types)):
        value = data_values.iat[i, j]
        rows.append({
            "Region": region,
            "Mode": mode,
            "Type": type_,
            "Branch": branch,
            "Value": value
        })

df_long = pd.DataFrame(rows)

# ==========================
# Step 4: Generate CSVs for each Type × Mode
# ==========================
unique_types = df_long["Type"].unique()
unique_modes = df_long["Mode"].unique()

for type_ in unique_types:
    for mode in unique_modes:
        # Filter data
        df_filtered = df_long[(df_long["Type"] == type_) & (df_long["Mode"] == mode)]
        
        # Pivot: rows=Region, columns=Branch
        df_pivot = df_filtered.pivot_table(
            index="Region",
            columns="Branch",
            values="Value",
            aggfunc='sum'
        ).reset_index()
        
        # Add Total column
        df_pivot["Total"] = df_pivot.iloc[:, 1:].sum(axis=1)
        
        # Optional: fill NaN with 0
        df_pivot.fillna(0, inplace=True)
        
        # Save CSV
        filename = f"{type_}_{mode}.csv".replace(" ", "_")
        df_pivot.to_csv(filename, index=False)
        print(f"Saved: {filename}")


/var/folders/lw/mxt4vf712k5_lslhrjcqqj700000gn/T/ipykernel_67244/2145665781.py:6: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254

Saved: Volume_Ground.csv
Saved: Volume_Air.csv
Saved: Billed_Wt_Ground.csv
Saved: Billed_Wt_Air.csv


In [3]:
import pandas as pd

# ==========================
# Step 1: List the exact CSV files
# ==========================
csv_files = [
    "Volume_Air.csv",
    "Volume_Ground.csv",
    "Billed_Wt_Air.csv",
    "Billed_Wt_Ground.csv"
]

# ==========================
# Step 2: Process each file
# ==========================
for file in csv_files:
    df = pd.read_csv(file)
    
    # Identify branch columns (all except 'Region' and 'Total')
    branch_cols = [col for col in df.columns if col not in ["Region", "Total"]]
    
    # Calculate percentage contribution per branch
    df_percentage = df.copy()
    for col in branch_cols:
        df_percentage[col] = df[col] / df["Total"] * 100
    
    # Keep 4 decimal places
    df_percentage[branch_cols] = df_percentage[branch_cols].round(4)
    
    # Drop the 'Total' column
    df_percentage = df_percentage.drop(columns=["Total"])
    
    # Save to new CSV
    new_filename = file.replace(".csv", "_percentage.csv")
    df_percentage.to_csv(new_filename, index=False)
    print(f"Saved: {new_filename}")

Saved: Volume_Air_percentage.csv
Saved: Volume_Ground_percentage.csv
Saved: Billed_Wt_Air_percentage.csv
Saved: Billed_Wt_Ground_percentage.csv


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

def process_percentage_csvs(csv_files, base_folder="bags"):
    """
    For a list of percentage CSV files:
    - Computes optimal branches per region using elbow method
    - Saves DataFrames, plots, and JSON in separate folder per CSV
    """
    os.makedirs(base_folder, exist_ok=True)

    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        branch_cols = [c for c in df.columns if c != "Region"]
        results = []

        # Folder for this CSV
        file_key = os.path.basename(csv_file).replace(".csv", "")
        csv_folder = os.path.join(base_folder, file_key)
        os.makedirs(csv_folder, exist_ok=True)

        branch_json = {}

        for idx, row in df.iterrows():
            region = row["Region"]
            # Sort branch percentages descending
            sorted_perc = row[branch_cols].sort_values(ascending=False)
            cumulative_perc = sorted_perc.cumsum().values
            num_branches = np.arange(1, len(sorted_perc)+1)

            # Elbow detection (farthest point from line)
            x1, y1 = cumulative_perc[0], num_branches[0]
            x2, y2 = cumulative_perc[-1], num_branches[-1]
            distances = np.abs((y2-y1)*cumulative_perc - (x2-x1)*num_branches + x2*y1 - y2*x1) / np.sqrt((y2-y1)**2 + (x2-x1)**2)
            elbow_idx = distances.argmax()
            optimal_percentage = cumulative_perc[elbow_idx]
            optimal_branches = num_branches[elbow_idx]

            results.append({
                "Region": region,
                "Optimal_Percentage": optimal_percentage,
                "Num_Branches": optimal_branches
            })

            # Branch names for this elbow
            optimal_branch_names = sorted_perc.index[:optimal_branches].tolist()
            branch_json[region] = optimal_branch_names

            # Plot
            plt.figure(figsize=(8,5))
            plt.plot(cumulative_perc, num_branches, marker='o', label='Cumulative % vs Branches')
            plt.scatter(optimal_percentage, optimal_branches, color='red', s=100, label='Elbow Point')
            plt.text(optimal_percentage, optimal_branches+0.5,
                     f"Pct={optimal_percentage:.4f}\nBranches={optimal_branches}",
                     color='red', fontsize=9, ha='center')
            plt.xlabel("Cumulative Percentage (%)")
            plt.ylabel("Number of Branches")
            plt.title(f"Region: {region} - {file_key}")
            plt.grid(True)
            plt.legend()
            
            # Save plot
            plot_filename = os.path.join(csv_folder, f"{region}_elbow.png")
            plt.savefig(plot_filename, bbox_inches='tight')
            plt.close()

        # Save DataFrame
        df_results = pd.DataFrame(results)
        df_filename = os.path.join(csv_folder, "optimal_branches.csv")
        df_results.to_csv(df_filename, index=False)

        # Save JSON
        json_filename = os.path.join(csv_folder, "optimal_branches.json")
        with open(json_filename, "w") as f:
            json.dump(branch_json, f, indent=4)

        print(f"Processed {csv_file}: saved CSV, plots, and JSON in {csv_folder}")


# ==========================
# Example usage with all 4 percentage CSVs
# ==========================
csv_files = [
    "Volume_Air_percentage.csv",
    "Volume_Ground_percentage.csv",
    "Billed_Wt_Air_percentage.csv",
    "Billed_Wt_Ground_percentage.csv"
]

process_percentage_csvs(csv_files, base_folder="bags")

Processed Volume_Air_percentage.csv: saved CSV, plots, and JSON in bags/Volume_Air_percentage
Processed Volume_Ground_percentage.csv: saved CSV, plots, and JSON in bags/Volume_Ground_percentage
Processed Billed_Wt_Air_percentage.csv: saved CSV, plots, and JSON in bags/Billed_Wt_Air_percentage
Processed Billed_Wt_Ground_percentage.csv: saved CSV, plots, and JSON in bags/Billed_Wt_Ground_percentage


In [5]:
# Mapping of region codes to their hub office codes.
region_code_to_hub_office = {
    'AMD': 'A01',   # AHMEDABAD
    'BLR': 'B10',   # BANGALORE
    'CHE': 'C20',   # CHENNAI
    'CJB': 'E01',   # COIMBATORE
    'HYD': 'H13',   # HYDERABAD
    'IDR': 'I01',   # INDORE REGIONAL OFFICE
    'HHPT': 'J10',  # HHPT REGIONAL OFFICE
    'CCU': 'K16',   # KOLKATA
    'GGN': 'N05',   # GGR REGIONAL OFFICE / NORTH DELHI RO / SOUTH DELHI RO
    'MUM': 'M10',   # MUMBAI
    'NDL': 'N05',   # NORTH DELHI RO
    'COK': 'O06',   # COCHIN
    'PNQ': 'P01',   # PUNE
    'JAI': 'Q05',   # JAIPUR
    'NGP': 'R01',   # NAGPUR RO
    'SDL': 'N05',   # SOUTH DELHI RO
    'PAT': 'T01',   # PATNA
    'VJA': 'V06',   # VIJAYAWADA
    'BBI': 'W53',   # BHUBANESHWAR
    'GAU': 'X03',   # GUWAHATI
    'EUP': None,    # Not mapped in provided mapping
    'WUP': None     # Not mapped in provided mapping
}